<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/-/blob/main/%E3%80%90note%E6%9C%AA%E5%85%AC%E9%96%8B%E3%80%91_%E5%B0%8F%E5%AD%A6%E7%94%9F%E5%90%91%E3%81%91%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E3%82%B9%E3%82%AF%E3%83%BC%E3%83%AB_AI%E3%82%AB%E3%83%AA%E3%82%AD%E3%83%A5%E3%83%A9%E3%83%A0%E4%BD%9C%E6%88%90%E3%83%84%E3%83%BC%E3%83%AB%E3%82%8C%E3%81%B9%E3%82%8B%E3%81%82%E3%81%92%EF%BC%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 🤖 Google Colab完全対応 ボタンクリック式プログラミングスクールAI
# 数字入力不要！クリックだけで操作

# 必要なライブラリのインストール
!pip install openai pandas matplotlib seaborn plotly ipywidgets

import openai
from openai import OpenAI
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import random
import re
from typing import List, Dict, Any
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, HTML, Output, Text, Textarea
import os
import threading
import time

# Google Colab Secretsのインポート
try:
    from google.colab import userdata
    COLAB_AVAILABLE = True
    print("✅ Google Colab環境を検出しました")
except ImportError:
    COLAB_AVAILABLE = False
    print("ℹ️ Google Colab環境ではありません")

# OpenAIクライアントのグローバル変数
client = None

def setup_openai_api():
    """OpenAI APIキーを安全に設定する関数"""
    global client
    print("🔐 OpenAI APIキーの設定中...")

    if COLAB_AVAILABLE:
        try:
            api_key = userdata.get('OPENAI_API_KEY')
            client = OpenAI(api_key=api_key)
            print("✅ Google Colab SecretsからAPIキーを読み込みました")
            return True
        except Exception as e:
            print(f"❌ Secretsからの読み込みに失敗: {e}")
            print("\n🔧 設定方法:")
            print("1. 左サイドバーの🔑（鍵アイコン）をクリック")
            print("2. 'Add new secret'をクリック")
            print("3. Name: OPENAI_API_KEY")
            print("4. Value: あなたのOpenAI APIキー")
            print("5. 'Notebook access'をONにする")
            return False
    else:
        api_key = os.getenv('OPENAI_API_KEY')
        if api_key:
            client = OpenAI(api_key=api_key)
            print("✅ 環境変数からAPIキーを読み込みました")
            return True
        else:
            print("❌ APIキーが見つかりません")
            return False

class ProgrammingSchoolGUI:
    """ボタンクリック式プログラミングスクールAIツール"""

    def __init__(self):
        """初期化"""
        self.current_step = "main_menu"
        self.selected_data = {}

        # 出力エリア
        self.output = Output()

        # データ定義
        self.grade_levels = {
            "低学年": {"ages": "6-8歳", "description": "視覚的で直感的な学習"},
            "中学年": {"ages": "9-10歳", "description": "論理的思考の基礎を育む"},
            "高学年": {"ages": "11-12歳", "description": "実践的なプログラミングの導入"}
        }

        self.curriculum_templates = {
            "基礎コース": "プログラミングの基本概念を楽しく学ぶ",
            "ゲーム制作コース": "ゲームを作りながらプログラミングを学ぶ",
            "ロボットコース": "ロボットを動かしてプログラミングを体験",
            "Webサイト制作コース": "自分のWebサイトを作ってみよう",
            "データサイエンス入門": "データを使って面白い発見をしよう"
        }

        self.main_functions = {
            "📚 カリキュラム生成": {
                "desc": "年齢に適した学習計画",
                "func": self.start_curriculum_generation
            },
            "📖 レッスンプラン作成": {
                "desc": "詳細な授業設計",
                "func": self.start_lesson_plan_creation
            },
            "🎯 クイズ生成": {
                "desc": "インタラクティブな問題作成",
                "func": self.start_quiz_generation
            },
            "🚀 プロジェクト提案": {
                "desc": "創造性を育む課題設計",
                "func": self.start_project_ideas
            },
            "📊 評価基準作成": {
                "desc": "成長を測る指標設定",
                "func": self.start_assessment_rubric
            },
            "📝 保護者レポート": {
                "desc": "進捗共有資料作成",
                "func": self.start_parent_communication
            }
        }

    def create_button(self, description, on_click, button_style='info', width='300px'):
        """統一されたボタンを作成"""
        button = Button(
            description=description,
            button_style=button_style,
            layout=widgets.Layout(width=width, height='40px', margin='5px')
        )
        button.on_click(on_click)
        return button

    def create_main_menu(self):
        """メインメニューを作成"""
        with self.output:
            clear_output(wait=True)
            print("🤖" + "="*60)
            print("     🎓 小学生プログラミングスクール AIアシスタント")
            print("            ボタンクリック版へようこそ！")
            print("="*63)
            print()
            print("下のボタンをクリックして機能を選択してください！")
            print()

        # メインメニューボタン
        buttons = []
        for title, info in self.main_functions.items():
            button = self.create_button(
                f"{title} - {info['desc']}",
                info['func'],
                'primary',
                '500px'
            )
            buttons.append(button)

        # 終了ボタン
        exit_button = self.create_button("🚪 終了", self.exit_app, 'danger', '200px')

        return VBox([
            HTML("<h2>🎯 何をお手伝いしましょうか？</h2>"),
            VBox(buttons),
            HBox([exit_button], layout=widgets.Layout(justify_content='center'))
        ])

    def start_curriculum_generation(self, b):
        """カリキュラム生成を開始"""
        self.selected_data = {"function": "curriculum"}
        self.show_grade_selection("カリキュラム生成")

    def start_lesson_plan_creation(self, b):
        """レッスンプラン作成を開始"""
        self.selected_data = {"function": "lesson_plan"}
        self.show_topic_input("レッスンプラン作成")

    def start_quiz_generation(self, b):
        """クイズ生成を開始"""
        self.selected_data = {"function": "quiz"}
        self.show_topic_input("クイズ生成")

    def start_project_ideas(self, b):
        """プロジェクトアイデア提案を開始"""
        self.selected_data = {"function": "project"}
        self.show_grade_selection("プロジェクトアイデア提案")

    def start_assessment_rubric(self, b):
        """評価基準作成を開始"""
        self.selected_data = {"function": "assessment"}
        self.show_skill_selection()

    def start_parent_communication(self, b):
        """保護者レポート作成を開始"""
        self.selected_data = {"function": "parent_report"}
        self.show_student_input()

    def show_grade_selection(self, function_name):
        """学年選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print(f"📚 {function_name} - 学年選択")
            print("="*40)
            print("対象学年をクリックで選択してください:")
            print()

        buttons = []
        for grade, info in self.grade_levels.items():
            button_text = f"{grade} ({info['ages']}) - {info['description']}"
            button = self.create_button(
                button_text,
                lambda b, g=grade: self.select_grade(g),
                'success',
                '500px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML(f"<h3>📚 {function_name} - 学年選択</h3>"),
            VBox(buttons),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def show_topic_input(self, function_name):
        """テーマ入力画面を表示"""
        with self.output:
            clear_output(wait=True)
            print(f"📝 {function_name} - テーマ入力")
            print("="*40)

        # テキスト入力
        if function_name == "レッスンプラン作成":
            default_text = "Scratchでゲーム作り"
            placeholder = "例: Scratchでゲーム作り"
        else:  # クイズ生成
            default_text = "プログラミングの基本"
            placeholder = "例: プログラミングの基本"

        text_input = Text(
            value=default_text,
            placeholder=placeholder,
            description='テーマ:',
            layout=widgets.Layout(width='400px')
        )

        # ボタン
        next_button = self.create_button(
            "次へ ➡️",
            lambda b: self.save_topic_and_continue(text_input.value),
            'primary',
            '200px'
        )

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML(f"<h3>📝 {function_name} - テーマ入力</h3>"),
            text_input,
            HBox([next_button, back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def save_topic_and_continue(self, topic):
        """テーマを保存して次のステップへ"""
        self.selected_data["topic"] = topic if topic.strip() else "デフォルトテーマ"

        if self.selected_data["function"] == "lesson_plan":
            self.show_grade_selection("レッスンプラン作成")
        else:  # quiz
            self.show_grade_selection("クイズ生成")

    def show_skill_selection(self):
        """スキル領域選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("📊 評価基準作成 - スキル領域選択")
            print("="*40)

        skills = [
            "プログラミング基礎知識",
            "論理的思考力",
            "創造性・表現力",
            "問題解決能力",
            "協働・コミュニケーション",
            "学習態度・意欲"
        ]

        buttons = []
        for skill in skills:
            button = self.create_button(
                skill,
                lambda b, s=skill: self.select_skill(s),
                'info',
                '400px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>📊 評価基準作成 - スキル領域選択</h3>"),
            VBox(buttons),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def show_student_input(self):
        """生徒情報入力画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("📝 保護者レポート作成 - 生徒情報入力")
            print("="*40)

        # 入力フィールド
        name_input = Text(
            value="田中太郎",
            placeholder="生徒名を入力",
            description='生徒名:',
            layout=widgets.Layout(width='300px')
        )

        progress_input = Textarea(
            value="Scratchの基本操作を習得し、簡単なアニメーションが作成できるようになりました",
            placeholder="学習進捗を入力してください",
            description='学習進捗:',
            layout=widgets.Layout(width='500px', height='80px')
        )

        goals_input = Textarea(
            value="より複雑なプログラムの作成と、チームでの協働プロジェクトに挑戦",
            placeholder="今後の学習目標を入力してください",
            description='今後の目標:',
            layout=widgets.Layout(width='500px', height='80px')
        )

        # ボタン
        next_button = self.create_button(
            "次へ ➡️",
            lambda b: self.save_student_info_and_continue(name_input.value, progress_input.value, goals_input.value),
            'primary',
            '200px'
        )

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>📝 保護者レポート作成 - 生徒情報入力</h3>"),
            name_input,
            progress_input,
            goals_input,
            HBox([next_button, back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def save_student_info_and_continue(self, name, progress, goals):
        """生徒情報を保存して次のステップへ"""
        self.selected_data.update({
            "student_name": name if name.strip() else "田中太郎",
            "progress": progress if progress.strip() else "基本操作を習得",
            "goals": goals if goals.strip() else "さらなるスキル向上"
        })
        self.show_grade_selection("保護者レポート作成")

    def select_grade(self, grade):
        """学年を選択"""
        self.selected_data["grade"] = grade

        # 機能に応じて次のステップへ
        if self.selected_data["function"] == "curriculum":
            self.show_course_selection()
        elif self.selected_data["function"] == "lesson_plan":
            self.show_duration_selection()
        elif self.selected_data["function"] == "quiz":
            self.show_question_count_selection()
        elif self.selected_data["function"] == "project":
            self.show_theme_selection()
        elif self.selected_data["function"] == "parent_report":
            self.execute_function()
        else:
            self.execute_function()

    def select_skill(self, skill):
        """スキルを選択"""
        self.selected_data["skill"] = skill
        self.show_grade_selection("評価基準作成")

    def show_course_selection(self):
        """コース選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("🎯 カリキュラム生成 - コース選択")
            print("="*40)

        buttons = []
        for course, description in self.curriculum_templates.items():
            button = self.create_button(
                f"{course} - {description}",
                lambda b, c=course: self.select_course(c),
                'info',
                '500px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>🎯 カリキュラム生成 - コース選択</h3>"),
            VBox(buttons),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_course(self, course):
        """コースを選択"""
        self.selected_data["course"] = course
        self.show_weeks_selection()

    def show_weeks_selection(self):
        """期間選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("⏰ カリキュラム生成 - 期間選択")
            print("="*40)

        weeks_options = [4, 6, 8, 12, 16, 20, 24]
        buttons = []

        for weeks in weeks_options:
            button = self.create_button(
                f"{weeks}週間",
                lambda b, w=weeks: self.select_weeks(w),
                'success',
                '150px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>⏰ カリキュラム生成 - 期間選択</h3>"),
            HBox(buttons[:4], layout=widgets.Layout(justify_content='center')),
            HBox(buttons[4:], layout=widgets.Layout(justify_content='center')),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_weeks(self, weeks):
        """期間を選択"""
        self.selected_data["weeks"] = weeks
        self.execute_function()

    def show_duration_selection(self):
        """授業時間選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("⏰ レッスンプラン作成 - 授業時間選択")
            print("="*40)

        durations = [30, 45, 60, 90]
        buttons = []

        for duration in durations:
            button = self.create_button(
                f"{duration}分",
                lambda b, d=duration: self.select_duration(d),
                'success',
                '150px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>⏰ レッスンプラン作成 - 授業時間選択</h3>"),
            HBox(buttons, layout=widgets.Layout(justify_content='center')),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_duration(self, duration):
        """授業時間を選択"""
        self.selected_data["duration"] = duration
        self.execute_function()

    def show_question_count_selection(self):
        """問題数選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("🔢 クイズ生成 - 問題数選択")
            print("="*40)

        counts = [3, 5, 8, 10]
        buttons = []

        for count in counts:
            button = self.create_button(
                f"{count}問",
                lambda b, c=count: self.select_question_count(c),
                'success',
                '150px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>🔢 クイズ生成 - 問題数選択</h3>"),
            HBox(buttons, layout=widgets.Layout(justify_content='center')),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_question_count(self, count):
        """問題数を選択"""
        self.selected_data["question_count"] = count
        self.execute_function()

    def show_theme_selection(self):
        """プロジェクトテーマ選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("🎨 プロジェクトアイデア提案 - テーマ選択")
            print("="*40)

        themes = [
            "ゲーム制作",
            "アート・デザイン",
            "ストーリーテリング",
            "科学・実験",
            "音楽・サウンド",
            "自由テーマ"
        ]

        buttons = []
        for theme in themes:
            button = self.create_button(
                theme,
                lambda b, t=theme: self.select_theme(t),
                'info',
                '300px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>🎨 プロジェクトアイデア提案 - テーマ選択</h3>"),
            VBox(buttons),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_theme(self, theme):
        """テーマを選択"""
        self.selected_data["theme"] = theme
        self.show_idea_count_selection()

    def show_idea_count_selection(self):
        """アイデア数選択画面を表示"""
        with self.output:
            clear_output(wait=True)
            print("💡 プロジェクトアイデア提案 - アイデア数選択")
            print("="*40)

        counts = [3, 5, 7, 10]
        buttons = []

        for count in counts:
            button = self.create_button(
                f"{count}個",
                lambda b, c=count: self.select_idea_count(c),
                'success',
                '150px'
            )
            buttons.append(button)

        back_button = self.create_button("🔙 戻る", self.back_to_main, 'warning', '200px')

        display(VBox([
            HTML("<h3>💡 プロジェクトアイデア提案 - アイデア数選択</h3>"),
            HBox(buttons, layout=widgets.Layout(justify_content='center')),
            HBox([back_button], layout=widgets.Layout(justify_content='center'))
        ]))

    def select_idea_count(self, count):
        """アイデア数を選択"""
        self.selected_data["idea_count"] = count
        self.execute_function()

    def execute_function(self):
        """選択された機能を実行"""
        with self.output:
            clear_output(wait=True)
            print("🤖 AI処理を実行中...")
            print("少しお待ちください... ☕")
            print()

        # プログレスバー風の表示
        progress_widget = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='処理中:',
            bar_style='info',
            style={'bar_color': '#4CAF50'},
            orientation='horizontal'
        )

        display(VBox([
            HTML("<h3>🤖 AI処理実行中...</h3>"),
            progress_widget
        ]))

        # 非同期でプログレスバーを更新
        def update_progress():
            for i in range(0, 101, 10):
                progress_widget.value = i
                time.sleep(0.3)

        # バックグラウンドでプログレスバー更新
        progress_thread = threading.Thread(target=update_progress)
        progress_thread.start()

        try:
            # 実際のAI処理を実行
            if self.selected_data["function"] == "curriculum":
                result = self.generate_curriculum()
            elif self.selected_data["function"] == "lesson_plan":
                result = self.create_lesson_plan()
            elif self.selected_data["function"] == "quiz":
                result = self.generate_quiz()
            elif self.selected_data["function"] == "project":
                result = self.create_project_ideas()
            elif self.selected_data["function"] == "assessment":
                result = self.create_assessment_rubric()
            elif self.selected_data["function"] == "parent_report":
                result = self.generate_parent_communication()
            else:
                result = {"error": "未知の機能です"}

            # プログレスバー完了を待つ
            progress_thread.join()

            # 結果を表示
            self.show_result(result)

        except Exception as e:
            progress_thread.join()
            with self.output:
                clear_output(wait=True)
                print(f"❌ エラーが発生しました: {str(e)}")

            self.show_back_to_main_button()

    def show_result(self, result):
        """結果を表示"""
        with self.output:
            clear_output(wait=True)

            if isinstance(result, dict) and "error" in result:
                print(f"❌ エラー: {result['error']}")
            else:
                print("🎉 処理完了！")
                print("="*50)

                if self.selected_data["function"] == "curriculum":
                    self.display_curriculum_result(result)
                elif self.selected_data["function"] == "lesson_plan":
                    self.display_lesson_plan_result(result)
                elif self.selected_data["function"] == "quiz":
                    self.display_quiz_result(result)
                elif self.selected_data["function"] == "project":
                    self.display_project_result(result)
                elif self.selected_data["function"] == "assessment":
                    self.display_assessment_result(result)
                elif self.selected_data["function"] == "parent_report":
                    self.display_parent_report_result(result)

        self.show_back_to_main_button()

    def display_curriculum_result(self, result):
        """カリキュラム結果を表示"""
        if isinstance(result, dict) and "course_title" in result:
            print(f"📖 コース名: {result.get('course_title', 'N/A')}")
            print(f"🎯 対象: {result.get('target_grade', 'N/A')}")
            print(f"⏰ 期間: {result.get('duration_weeks', 'N/A')}週間")
            print(f"📋 学習目標数: {len(result.get('learning_objectives', []))}")
            print(f"📚 週次レッスン数: {len(result.get('weekly_lessons', []))}")
            print()

            print("📋 学習目標:")
            for i, obj in enumerate(result.get('learning_objectives', []), 1):
                print(f"  {i}. {obj}")
            print()

            print("📚 週次レッスン:")
            for lesson in result.get('weekly_lessons', [])[:5]:  # 最初の5週間を表示
                print(f"  第{lesson.get('week', 'N/A')}週: {lesson.get('title', 'N/A')}")

            if len(result.get('weekly_lessons', [])) > 5:
                print(f"  ... 他{len(result.get('weekly_lessons', [])) - 5}週間")
        else:
            print("結果の表示に失敗しました")

    def display_lesson_plan_result(self, result):
        """レッスンプラン結果を表示"""
        if isinstance(result, dict) and "lesson_title" in result:
            print(f"📖 レッスン名: {result.get('lesson_title', 'N/A')}")
            print(f"🎯 対象: {result.get('target_grade', 'N/A')}")
            print(f"⏰ 時間: {result.get('duration_minutes', 'N/A')}分")
            print()

            print("📚 授業構成:")
            for phase in result.get('lesson_structure', []):
                print(f"  {phase.get('phase', 'N/A')} ({phase.get('duration_minutes', 'N/A')}分)")
                for activity in phase.get('activities', [])[:2]:  # 最初の2つの活動を表示
                    print(f"    - {activity}")
        else:
            print("結果の表示に失敗しました")

    def display_quiz_result(self, result):
        """クイズ結果を表示"""
        if isinstance(result, list) and len(result) > 0 and "question_text" in result[0]:
            print(f"🎯 {len(result)}問のクイズが完成しました！")
            print()

            for i, question in enumerate(result[:3], 1):  # 最初の3問を表示
                print(f"【問題 {i}】")
                print(question.get('question_text', 'N/A'))
                print(f"難易度: {question.get('difficulty', 'N/A')}")
                print()

            if len(result) > 3:
                print(f"... 他{len(result) - 3}問")
        else:
            print("結果の表示に失敗しました")

    def display_project_result(self, result):
        """プロジェクト結果を表示"""
        if isinstance(result, list) and len(result) > 0 and "project_title" in result[0]:
            print(f"🚀 {len(result)}個のプロジェクトアイデアが完成しました！")
            print()

            for i, project in enumerate(result, 1):
                print(f"💡 アイデア {i}: {project.get('project_title', 'N/A')}")
                print(f"   📝 説明: {project.get('description', 'N/A')}")
                print(f"   ⭐ 難易度: {project.get('difficulty', 'N/A')}")
                print(f"   ⏰ 推定時間: {project.get('estimated_time', 'N/A')}")
                print()
        else:
            print("結果の表示に失敗しました")

    def display_assessment_result(self, result):
        """評価基準結果を表示"""
        if isinstance(result, dict) and "skill_area" in result:
            print(f"📊 評価領域: {result.get('skill_area', 'N/A')}")
            print(f"🎯 対象: {result.get('target_grade', 'N/A')}")
            print()

            criteria = result.get('evaluation_criteria', [])
            print(f"📋 評価項目数: {len(criteria)}")
            for criterion in criteria:
                print(f"  - {criterion.get('criterion', 'N/A')}")
        else:
            print("結果の表示に失敗しました")

    def display_parent_report_result(self, result):
        """保護者レポート結果を表示"""
        if isinstance(result, str) and len(result) > 10:
            print("📝 保護者向けレポート:")
            print("-" * 40)
            print(result)
        else:
            print("結果の表示に失敗しました")

    def show_back_to_main_button(self):
        """メインメニューに戻るボタンを表示"""
        back_button = self.create_button("🏠 メインメニューに戻る", self.back_to_main, 'primary', '300px')
        display(HBox([back_button], layout=widgets.Layout(justify_content='center')))

    def back_to_main(self, b):
        """メインメニューに戻る"""
        self.selected_data = {}
        with self.output:
            clear_output(wait=True)
        display(self.create_main_menu())

    def exit_app(self, b):
        """アプリケーションを終了"""
        with self.output:
            clear_output(wait=True)
            print("👋 ご利用ありがとうございました！")
            print("また素敵な学習体験を作りましょう！")
            print()
            print("新しく始める場合は、もう一度 start_gui() を実行してください。")

    # AI機能実装メソッド（OpenAI v1対応）
    def generate_curriculum(self):
        """カリキュラム生成"""
        try:
            prompt = f"""
            小学生向けプログラミングスクールの{self.selected_data['weeks']}週間カリキュラムを作成してください。

            対象: {self.selected_data['grade']}
            コース: {self.selected_data['course']}
            期間: {self.selected_data['weeks']}週間

            以下の形式でJSONを出力してください：
            {{
                "course_title": "コース名",
                "target_grade": "{self.selected_data['grade']}",
                "duration_weeks": {self.selected_data['weeks']},
                "learning_objectives": ["学習目標1", "学習目標2", "学習目標3"],
                "weekly_lessons": [
                    {{
                        "week": 1,
                        "title": "レッスンタイトル",
                        "objectives": ["目標1", "目標2"],
                        "activities": ["活動1", "活動2"],
                        "materials": ["必要な道具"],
                        "homework": "宿題内容",
                        "assessment": "評価方法"
                    }}
                ]
            }}

            要件:
            - 小学生が楽しめる内容
            - 段階的な難易度設定
            - 実践的な活動を含む
            """

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )

            result_text = response.choices[0].message.content
            json_match = re.search(r'\{.*\}', result_text, re.DOTALL)

            if json_match:
                return json.loads(json_match.group())
            else:
                return {"error": "JSONフォーマットの解析に失敗しました"}

        except Exception as e:
            return {"error": f"カリキュラム生成エラー: {str(e)}"}

    def create_lesson_plan(self):
        """レッスンプラン作成"""
        try:
            prompt = f"""
            小学生向けプログラミング授業の詳細なレッスンプランを作成してください。

            テーマ: {self.selected_data['topic']}
            対象: {self.selected_data['grade']}
            時間: {self.selected_data['duration']}分

            以下の形式でJSONを出力してください：
            {{
                "lesson_title": "レッスンタイトル",
                "duration_minutes": {self.selected_data['duration']},
                "target_grade": "{self.selected_data['grade']}",
                "learning_objectives": ["目標1", "目標2"],
                "materials_needed": ["必要な材料"],
                "lesson_structure": [
                    {{
                        "phase": "導入",
                        "duration_minutes": 10,
                        "activities": ["活動内容"],
                        "teacher_notes": "指導のポイント"
                    }}
                ],
                "assessment_criteria": ["評価基準"],
                "safety_notes": ["安全上の注意"]
            }}
            """

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )

            result_text = response.choices[0].message.content
            json_match = re.search(r'\{.*\}', result_text, re.DOTALL)

            if json_match:
                return json.loads(json_match.group())
            else:
                return {"error": "JSONフォーマットの解析に失敗しました"}

        except Exception as e:
            return {"error": f"レッスンプラン作成エラー: {str(e)}"}

    def generate_quiz(self):
        """クイズ生成"""
        try:
            prompt = f"""
            小学生向けプログラミングクイズを作成してください。

            テーマ: {self.selected_data['topic']}
            対象: {self.selected_data['grade']}
            問題数: {self.selected_data['question_count']}問

            以下の形式でJSONを出力してください：
            [
                {{
                    "question_number": 1,
                    "question_text": "問題文",
                    "question_type": "multiple_choice",
                    "options": ["選択肢1", "選択肢2", "選択肢3", "選択肢4"],
                    "correct_answer": "正解",
                    "explanation": "解説",
                    "difficulty": "easy"
                }}
            ]
            """

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )

            result_text = response.choices[0].message.content
            json_match = re.search(r'\[.*\]', result_text, re.DOTALL)

            if json_match:
                return json.loads(json_match.group())
            else:
                return [{"error": "JSONフォーマットの解析に失敗しました"}]

        except Exception as e:
            return [{"error": f"クイズ生成エラー: {str(e)}"}]

    def create_project_ideas(self):
        """プロジェクトアイデア作成"""
        try:
            prompt = f"""
            小学生向けプログラミングプロジェクトのアイデアを考えてください。

            対象: {self.selected_data['grade']}
            テーマ: {self.selected_data['theme']}
            アイデア数: {self.selected_data['idea_count']}個

            以下の形式でJSONを出力してください：
            [
                {{
                    "project_title": "プロジェクト名",
                    "description": "プロジェクトの説明",
                    "difficulty": "初級",
                    "estimated_time": "推定制作時間",
                    "tools_needed": ["必要なツール"],
                    "learning_outcomes": ["学習成果1", "学習成果2"]
                }}
            ]
            """

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.8
            )

            result_text = response.choices[0].message.content
            json_match = re.search(r'\[.*\]', result_text, re.DOTALL)

            if json_match:
                return json.loads(json_match.group())
            else:
                return [{"error": "JSONフォーマットの解析に失敗しました"}]

        except Exception as e:
            return [{"error": f"プロジェクトアイデア生成エラー: {str(e)}"}]

    def create_assessment_rubric(self):
        """評価基準作成"""
        try:
            prompt = f"""
            小学生向けプログラミング学習の評価ルーブリックを作成してください。

            評価領域: {self.selected_data['skill']}
            対象: {self.selected_data['grade']}

            以下の形式でJSONを出力してください：
            {{
                "skill_area": "{self.selected_data['skill']}",
                "target_grade": "{self.selected_data['grade']}",
                "evaluation_criteria": [
                    {{
                        "criterion": "評価項目名",
                        "levels": {{
                            "excellent": {{"description": "優秀レベル", "indicators": ["指標1"]}},
                            "good": {{"description": "良好レベル", "indicators": ["指標1"]}},
                            "developing": {{"description": "発達中レベル", "indicators": ["指標1"]}},
                            "beginning": {{"description": "初心者レベル", "indicators": ["指標1"]}}
                        }}
                    }}
                ]
            }}
            """

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.6
            )

            result_text = response.choices[0].message.content
            json_match = re.search(r'\{.*\}', result_text, re.DOTALL)

            if json_match:
                return json.loads(json_match.group())
            else:
                return {"error": "JSONフォーマットの解析に失敗しました"}

        except Exception as e:
            return {"error": f"評価ルーブリック作成エラー: {str(e)}"}

    def generate_parent_communication(self):
        """保護者レポート生成"""
        try:
            prompt = f"""
            小学生の保護者向けプログラミング学習進捗レポートを作成してください。

            生徒名: {self.selected_data['student_name']}
            学年: {self.selected_data['grade']}
            進捗要約: {self.selected_data['progress']}
            今後の目標: {self.selected_data['goals']}

            温かみのあるレポートを作成してください。
            以下の要素を含む：
            - 生徒の頑張りを認める内容
            - 具体的な成長ポイント
            - 家庭でできるサポート方法
            - 今後の学習計画
            """

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )

            return response.choices[0].message.content

        except Exception as e:
            return f"保護者向けレポート作成エラー: {str(e)}"

    def start(self):
        """GUIを開始"""
        display(self.output)
        display(self.create_main_menu())

# GUI起動関数
def start_gui():
    """ボタンクリック式GUIを開始"""
    print("🚀 ボタンクリック式プログラミングスクールAIを起動中...")
    print("=" * 60)

    # APIキーの設定
    if setup_openai_api():
        print("✅ APIキーの設定が完了しました")

        # API接続テスト
        print("🧪 API接続テスト中...")
        try:
            test_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": "Hello"}],
                max_tokens=5
            )
            print("✅ API接続確認完了")
            print("\n🎉 すべての準備が完了しました！")
            print("下にボタンが表示されます。クリックして操作してください。")
            print("=" * 60)

            # GUI開始
            gui = ProgrammingSchoolGUI()
            gui.start()

        except Exception as e:
            print(f"❌ API接続に失敗しました: {e}")
            print("APIキーまたはネットワーク接続を確認してください")
    else:
        print("❌ APIキーの設定に失敗しました")
        print("Google Colab Secretsの設定を確認してください")

# 使用方法の表示
print("🤖 Google Colab完全対応 ボタンクリック式プログラミングスクールAI")
print("=" * 70)
print()
print("📋 このツールでできること:")
print("1. 📚 カリキュラム生成 - 年齢に適した学習計画")
print("2. 📖 レッスンプラン作成 - 詳細な授業設計")
print("3. 🎯 クイズ生成 - インタラクティブな問題作成")
print("4. 🚀 プロジェクト提案 - 創造性を育む課題設計")
print("5. 📊 評価基準作成 - 成長を測る指標設定")
print("6. 📝 保護者レポート - 進捗共有資料作成")
print()
print("🔧 使用方法:")
print("1. Google Colab Secretsに'OPENAI_API_KEY'を設定")
print("2. start_gui() を実行")
print("3. 表示されるボタンをクリックして操作")
print("4. すべてマウスクリックで完結！")
print()
print("💡 今すぐ開始:")
print("start_gui()")
print()
print("✨ 特徴:")
print("- キーボード入力不要！")
print("- すべてボタンクリックで操作")
print("- 美しいプログレスバー付き")
print("- エラー処理完備")

print("\n" + "="*70)
print("✅ ボタンクリック式GUI読み込み完了！")
print("start_gui() で今すぐ開始")
print("="*70)

# まず診断テストを実行
quick_test()



✅ Google Colab環境を検出しました
🤖 Google Colab完全対応 ボタンクリック式プログラミングスクールAI

📋 このツールでできること:
1. 📚 カリキュラム生成 - 年齢に適した学習計画
2. 📖 レッスンプラン作成 - 詳細な授業設計
3. 🎯 クイズ生成 - インタラクティブな問題作成
4. 🚀 プロジェクト提案 - 創造性を育む課題設計
5. 📊 評価基準作成 - 成長を測る指標設定
6. 📝 保護者レポート - 進捗共有資料作成

🔧 使用方法:
1. Google Colab Secretsに'OPENAI_API_KEY'を設定
2. start_gui() を実行
3. 表示されるボタンをクリックして操作
4. すべてマウスクリックで完結！

💡 今すぐ開始:
start_gui()

✨ 特徴:
- キーボード入力不要！
- すべてボタンクリックで操作
- 美しいプログレスバー付き
- エラー処理完備

✅ ボタンクリック式GUI読み込み完了！
start_gui() で今すぐ開始


NameError: name 'quick_test' is not defined

In [6]:
#まず診断テストを実行※コメントアウトして実行してください
#quick_test()


#実行
start_improved_gui()

NameError: name 'start_improved_gui' is not defined

In [7]:
#まず診断テストを実行※コメントアウトして実行してください
# quick_test() # This function is not defined

#実行
start_gui() # Changed from start_improved_gui()

🚀 ボタンクリック式プログラミングスクールAIを起動中...
🔐 OpenAI APIキーの設定中...
✅ Google Colab SecretsからAPIキーを読み込みました
✅ APIキーの設定が完了しました
🧪 API接続テスト中...
✅ API接続確認完了

🎉 すべての準備が完了しました！
下にボタンが表示されます。クリックして操作してください。


Output()